# Tasks Workflow:

### Dependencies loading

In [42]:
!pip install xgboostlss

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   --------- ------------------------------ 0.8/3.2 MB 5.6 MB/s eta 0:00:01
   --------------------------------- ------ 2.6/3.2 MB 7.9 MB/s eta 0:00:01
   ---------------------------------------- 3.2/3.2 MB 7.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 756.0/756.0 kB 7.8 MB/s eta 0:00:00


In [175]:
import os
import pickle
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
)

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import (
    AdaBoostRegressor,
    GradientBoostingRegressor,
    HistGradientBoostingRegressor,
)

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

from xgboostlss.model import XGBoostLSS
from xgboostlss.distributions.Gaussian import Gaussian

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 150)

### 1. Data loading

In [5]:
base_url = "https://raw.githubusercontent.com/anlan-chen/Gradient_Boosting_Task/main/Data/"

train_fe_url    = base_url + "train_fe.csv"
test_fe_url     = base_url + "test_fe.csv"
feature_rank_url = base_url + "feature_ranking.xlsx"


In [9]:
train_fe = pd.read_csv(train_fe_url)
train_fe.head()

,Unnamed: 0,Ticker,Nazwa2,rok,ta,txt,pi,str,xrd,ni,ppent,intant,dlc,dltt,capex,revenue,cce,adv,etr,diff,roa,lev,intan,rd,ppe,sale,cash_holdings,adv_expenditure,capex2,cfc,dta,capex2_scaled,y_v2x_polyarchy,y_e_p_polity,y_BR_Democracy,WB_GDPgrowth,WB_GDPpc,WB_Inflation,rr_per_country,rr_per_sector,sektor_consumer discretionary,sektor_consumer staples,sektor_energy,sektor_health care,sektor_industrials,sektor_materials,sektor_real estate,sektor_technology,sektor_utilities,gielda_2,gielda_3,gielda_4,gielda_5,ta_log,"txt_cat_(-63.011, -34.811]","txt_cat_(-34.811, 0.488]","txt_cat_(0.488, 24.415]","txt_cat_(24.415, 25.05]","txt_cat_(25.05, 308.55]","txt_cat_(308.55, 327.531]","txt_cat_(327.531, inf]","pi_cat_(-8975.0, -1.523]","pi_cat_(-1.523, 157.119]","pi_cat_(157.119, 465.9]","pi_cat_(465.9, 7875.5]","pi_cat_(7875.5, 8108.5]","pi_cat_(8108.5, inf]","str_cat_(0.0875, 0.192]","str_cat_(0.192, 0.28]","str_cat_(0.28, inf]",xrd_exists,ni_profit,ni_profit_20000,ppent_sqrt,intant_sqrt,"dlc_cat_(42.262, 176.129]","dlc_cat_(176.129, 200.9]","dlc_cat_(200.9, inf]","dltt_cat_(39.38, 327.85]","dltt_cat_(327.85, 876.617]","dltt_cat_(876.617, inf]","capex_cat_(7.447, 79.55]","capex_cat_(79.55, 5451.0]","capex_cat_(5451.0, inf]","revenue_cat_(0.174, 1248.817]","revenue_cat_(1248.817, 4233.587]","revenue_cat_(4233.587, inf]","cce_cat_(5.619, 63.321]","cce_cat_(63.321, inf]","adv_cat_(0.3, 874.5]","adv_cat_(874.5, inf]",diff_positive,roa_clip,lev_sqrt,intan_pow2,rd_sqrt,ppe_clip,cash_holdings_sqrt,adv_expenditure_positive,diff_dta,cfc_dta,etr_y_past,etr_y_ma,diff_ma,roa_ma,lev_ma,intan_ma,ppe_ma,sale_ma,cash_holdings_ma,roa_past,lev_past,intan_past,ppe_past,sale_past,cash_holdings_past
0,0,11B PW Equity,11 bit studios SA,2005,21.127613,1.24185,6.329725,0.19,0.0,5.0879,0.276275,4.1959,0.0,0.0,2.223413,11.873301,12.142975,0.0,0.196193,-0.006193,0.240818,0.0,0.198598,0.0,0.013076,0.445954,0.574744,0.0,8.047824,0,0,0.007469,0.877,10,1,3.493668,8021.003655,2.183799,0.334868,0.030010,0,0,0,0,0,0,0,0,0,0,0,0,0,3.050581,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0.525619,2.04839,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0.240818,0.0,0.039441,0.0,0.013076,0.758119,0,-0.0,0,0.196193,0.196193,-0.006193,0.240818,0.0,0.198598,0.013076,0.445954,0.574744,0.240818,0.0,0.198598,0.013076,0.445954,0.574744
1,1,11B PW Equity,11 bit studios SA,2006,21.127613,1.24185,6.329725,0.19,0.0,5.0879,0.276275,4.1959,0.0,0.0,2.223413,11.873301,12.142975,0.0,0.196193,-0.006193,0.240818,0.0,0.198598,0.0,0.013076,0.445954,0.574744,0.0,8.047824,0,0,0.007469,0.884,10,1,6.179641,9038.730847,1.284694,0.421396,0.136194,0,0,0,0,0,0,0,0,0,0,0,0,0,3.050581,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0.525619,2.04839,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0.240818,0.0,0.039441,0.0,0.013076,0.758119,0,-0.0,0,0.196193,0.196193,-0.006193,0.240818,0.0,0.198598,0.013076,0.445954,0.574744,0.240818,0.0,0.198598,0.013076,0.445954,0.574744
2,2,11B PW Equity,11 bit studios SA,2007,21.127613,1.24185,6.329725,0.19,0.0,5.0879,0.276275,4.1959,0.0,0.0,2.223413,11.873301,12.142975,0.0,0.196193,-0.006193,0.240818,0.0,0.198598,0.0,0.013076,0.445954,0.574744,0.0,8.047824,0,0,0.007469,0.887,10,1,7.034828,11255.436700,2.458743,0.104220,0.035493,0,0,0,0,0,0,0,0,0,0,0,0,0,3.050581,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0.525619,2.04839,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0.240818,0.0,0.039441,0.0,0.013076,0.758119,0,-0.0,0,0.196193,0.196193,-0.006193,0.240818,0.0,0.198598,0.013076,0.445954,0.574744,0.240818,0.0,0.198598,0.013076,0.445954,0.574744
3,3,11B PW Equity,11 bit studios SA,2008,21.127613,1.24185,6.329725,0.19,0.0,5.0879,0.276275,4.1959,0.0,0.0,2.223413,11.873301,12.142975,0.0,0.196193,-0.006193,0.240818,0.0,0.198598,0.0,0.013076,0.445954,0.574744,0.0,8.047824,0,0,0.007469,0.869,10,1,4.249609,14001.446880,4.164972,-0.511178,-0.384695,0,0,0,0,0,0,0,0,0,0,0,0,0,3.050581,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0.525619,2.04839,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0.240818,0.0,0.039441,0.0,0.013076,0.758119,0,-0.0,0,0.196193,0.196193,-0.00

In [10]:
test_fe = pd.read_csv(test_fe_url)
test_fe.head()

,Unnamed: 0,Ticker,Nazwa2,rok,ta,txt,pi,str,xrd,ni,ppent,intant,dlc,dltt,capex,revenue,cce,adv,etr,diff,roa,lev,intan,rd,ppe,sale,cash_holdings,adv_expenditure,capex2,cfc,dta,capex2_scaled,y_v2x_polyarchy,y_e_p_polity,y_BR_Democracy,WB_GDPgrowth,WB_GDPpc,WB_Inflation,rr_per_country,rr_per_sector,sektor_consumer discretionary,sektor_consumer staples,sektor_energy,sektor_health care,sektor_industrials,sektor_materials,sektor_real estate,sektor_technology,sektor_utilities,gielda_2,gielda_3,gielda_4,gielda_5,ta_log,"txt_cat_(-63.011, -34.811]","txt_cat_(-34.811, 0.488]","txt_cat_(0.488, 24.415]","txt_cat_(24.415, 25.05]","txt_cat_(25.05, 308.55]","txt_cat_(308.55, 327.531]","txt_cat_(327.531, inf]","pi_cat_(-8975.0, -1.523]","pi_cat_(-1.523, 157.119]","pi_cat_(157.119, 465.9]","pi_cat_(465.9, 7875.5]","pi_cat_(7875.5, 8108.5]","pi_cat_(8108.5, inf]","str_cat_(0.0875, 0.192]","str_cat_(0.192, 0.28]","str_cat_(0.28, inf]",xrd_exists,ni_profit,ni_profit_20000,ppent_sqrt,intant_sqrt,"dlc_cat_(42.262, 176.129]","dlc_cat_(176.129, 200.9]","dlc_cat_(200.9, inf]","dltt_cat_(39.38, 327.85]","dltt_cat_(327.85, 876.617]","dltt_cat_(876.617, inf]","capex_cat_(7.447, 79.55]","capex_cat_(79.55, 5451.0]","capex_cat_(5451.0, inf]","revenue_cat_(0.174, 1248.817]","revenue_cat_(1248.817, 4233.587]","revenue_cat_(4233.587, inf]","cce_cat_(5.619, 63.321]","cce_cat_(63.321, inf]","adv_cat_(0.3, 874.5]","adv_cat_(874.5, inf]",diff_positive,roa_clip,lev_sqrt,intan_pow2,rd_sqrt,ppe_clip,cash_holdings_sqrt,adv_expenditure_positive,diff_dta,cfc_dta,etr_y_past,etr_y_ma,diff_ma,roa_ma,lev_ma,intan_ma,ppe_ma,sale_ma,cash_holdings_ma,roa_past,lev_past,intan_past,ppe_past,sale_past,cash_holdings_past
0,11,11B PW Equity,11 bit studios SA,2016,45.649399,2.8015,15.730800,0.19,0.0,12.929300,0.7795,7.921200,0.00000,0.000000,5.408,27.015800,29.114000,0.0,0.214678,0.011910,0.283230,0.000000,0.173523,0.000000,0.017076,0.464872,0.637774,0.0,6.937781,1,0,0.006439,0.837,10,1,3.063908,12431.57542,-0.664767,0.132444,0.098435,0,0,0,0,0,0,0,0,0,0,0,0,0,3.820990,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0.882893,2.814463,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0.283230,0.000000,0.030110,0.000000,0.017076,0.798608,0,0.000000,0,0.178090,0.195133,-0.011960,0.390833,0.000000,0.130887,0.009081,0.539652,0.527618,0.352416,0.000000,0.194652,0.013346,0.534162,0.699963
1,23,1AT PW Equity,Atal SA/Poland,2016,1513.552979,15.9150,109.042999,0.19,0.0,89.442001,7.4800,63.091999,293.81601,217.255997,1.205,506.839996,187.462997,0.0,0.169561,0.044048,0.059094,0.337664,0.041685,0.000000,0.004942,0.288832,0.123856,0.0,0.161096,1,0,0.000150,0.837,10,1,3.063908,12431.57542,-0.664767,0.132444,0.046627,1,0,0,0,0,0,0,0,0,0,0,0,0,7.322215,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,2.734959,7.943047,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0.059094,0.581088,0.001738,0.000000,0.004942,0.351932,0,0.000000,0,0.145952,0.072976,0.253131,0.105069,0.208895,0.066162,0.007559,0.360836,0.100509,0.039084,0.196057,0.052127,0.006567,0.171832,0.107148
2,35,4FM PW Equity,4Fun Media SA,2016,41.662998,0.3202,1.489200,0.19,0.0,1.346000,5.6048,21.440599,0.01100,0.109600,1.283,38.155399,0.920100,0.0,0.138719,-0.025015,0.032307,0.002895,0.514620,0.000000,0.134527,0.650141,0.022084,0.0,0.228911,1,0,0.000212,0.837,10,1,3.063908,12431.57542,-0.664767,0.132444,0.098435,0,0,0,0,0,0,0,0,0,0,0,0,0,3.729613,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,2.367446,4.630399,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.032307,0.053802,0.264833,0.000000,0.134527,0.148608,0,-0.000000,0,0.215015,0.201590,0.004541,-0.055405,0.007222,0.496920,0.087819,0.470287,0.056824,0.067262,0.014443,0.564584,0.152826,0.469210,0.042710
3,47,AAL LN Equity,Anglo American PLC,2016,50149.000000,698.0000,2624.000000,0.20,1594.0,28719.000000,63.0000,3217.000000,11363.00000,1806.000000,2418.000,21378.000000,6051.000000,0.0,0.262670,-0.066006,0.572673,0.262597,0.064149,0.031785,0.001256,0.355076,0.120660,0.0,38.380952,1,1,0.035620,0.878,10,1,2.230000,42098.92045,0.491749,0.094914,0.145205,0,0,0,0,0,1,

In [11]:
feature_rank = pd.read_excel(feature_rank_url)
feature_rank.head()

,Unnamed: 0,mi_score,sign_fscore,sign_fscore_0_1,corr,EN_coef,boruta_rank
0,rok,0.032073,1.179353e-01,0,-0.032669,0.000000e+00,19
1,ta,0.582922,1.464884e-03,1,0.267340,-1.404307e-07,49
2,txt,0.633067,5.246456e-13,1,0.368732,1.466269e-05,1
3,pi,0.608157,8.614688e-12,1,0.299593,8.453656e-06,3
4,str,0.293955,1.578384e-46,1,0.372870,NaN,9


### 2. Feature engineering for models

In [45]:
# Set target variable
target_col = "etr"

# sort by year (rok) for time-series style CV 
train_fe = train_fe.sort_values(by="rok").reset_index(drop=True)
test_fe  = test_fe.sort_values(by="rok").reset_index(drop=True)

print("dtypes of train_fe:")
display(train_fe.dtypes.head(20))


dtypes of train_fe:


Unnamed: 0      int64
Ticker         object
Nazwa2         object
rok             int64
ta            float64
txt           float64
pi            float64
str           float64
xrd           float64
ni            float64
ppent         float64
intant        float64
dlc           float64
dltt          float64
capex         float64
revenue       float64
cce           float64
adv           float64
etr           float64
diff          float64
dtype: object

In [50]:
# Check for missing values (NA)
print("Missing values in train_fe (top 20):")
display(train_fe.isna().sum().sort_values(ascending=False).head(20))

print("Missing values in test_fe (top 20):")
display(test_fe.isna().sum().sort_values(ascending=False).head(20))


Missing values in train_fe (top 20):


Unnamed: 0                          0
ppent_sqrt                          0
revenue_cat_(1248.817, 4233.587]    0
revenue_cat_(0.174, 1248.817]       0
capex_cat_(5451.0, inf]             0
capex_cat_(79.55, 5451.0]           0
capex_cat_(7.447, 79.55]            0
dltt_cat_(876.617, inf]             0
dltt_cat_(327.85, 876.617]          0
dltt_cat_(39.38, 327.85]            0
dlc_cat_(200.9, inf]                0
dlc_cat_(176.129, 200.9]            0
dlc_cat_(42.262, 176.129]           0
intant_sqrt                         0
ni_profit_20000                     0
Ticker                              0
ni_profit                           0
xrd_exists                          0
str_cat_(0.28, inf]                 0
str_cat_(0.192, 0.28]               0
dtype: int64

Missing values in test_fe (top 20):


Unnamed: 0                          0
ppent_sqrt                          0
revenue_cat_(1248.817, 4233.587]    0
revenue_cat_(0.174, 1248.817]       0
capex_cat_(5451.0, inf]             0
capex_cat_(79.55, 5451.0]           0
capex_cat_(7.447, 79.55]            0
dltt_cat_(876.617, inf]             0
dltt_cat_(327.85, 876.617]          0
dltt_cat_(39.38, 327.85]            0
dlc_cat_(200.9, inf]                0
dlc_cat_(176.129, 200.9]            0
dlc_cat_(42.262, 176.129]           0
intant_sqrt                         0
ni_profit_20000                     0
Ticker                              0
ni_profit                           0
xrd_exists                          0
str_cat_(0.28, inf]                 0
str_cat_(0.192, 0.28]               0
dtype: int64

In [51]:
# If missing values, fill in using the median.
numeric_cols = train_fe.select_dtypes(include=[np.number]).columns

for col in numeric_cols:
    if train_fe[col].isna().sum() > 0 or test_fe[col].isna().sum() > 0:
        median_val = train_fe[col].median()
        train_fe[col] = train_fe[col].fillna(median_val)
        test_fe[col]  = test_fe[col].fillna(median_val)


In [52]:
# Extract target vector
y_train = train_fe[target_col].values
y_test  = test_fe[target_col].values

### 3. Feature Selection

In [56]:
feature_rank = pd.read_excel(feature_rank_url)
feature_rank.head()


,Unnamed: 0,mi_score,sign_fscore,sign_fscore_0_1,corr,EN_coef,boruta_rank
0,rok,0.032073,1.179353e-01,0,-0.032669,0.000000e+00,19
1,ta,0.582922,1.464884e-03,1,0.267340,-1.404307e-07,49
2,txt,0.633067,5.246456e-13,1,0.368732,1.466269e-05,1
3,pi,0.608157,8.614688e-12,1,0.299593,8.453656e-06,3
4,str,0.293955,1.578384e-46,1,0.372870,NaN,9


In [57]:
feature_rank.columns

Index(['Unnamed: 0', 'mi_score', 'sign_fscore', 'sign_fscore_0_1', 'corr',
       'EN_coef', 'boruta_rank'],
      dtype='object')

### 3.1 Initial feature selection using Boruta ranking

In [176]:
feature_rank = pd.read_excel(feature_rank_url, index_col=0)

# Sort by boruta_rank in ascending order (lower rank is more important)
feature_rank_sorted = feature_rank.sort_values(by="boruta_rank", ascending=True)

feature_names_sorted = feature_rank_sorted.index.tolist()

# Construct an initial feature subset based on Boruta ranking (top 20, 30, 50 features, etc.)
feature_sets = {
    "top20": feature_names_sorted[:20],
    "top30": feature_names_sorted[:30],
    "top50": feature_names_sorted[:50],
}

print("Initial feature sets from Boruta ranking:")
for name, cols in feature_sets.items():
    print(f"{name:10s}: {len(cols)} features")


Initial feature sets from Boruta ranking:
top20     : 20 features
top30     : 30 features
top50     : 50 features


### 3.2 Model-based feature importance from Gradient Boosting

In [177]:
# 先用 Boruta Top50 构造训练/测试特征矩阵
def get_X(df, feature_set_name):
    cols = feature_sets[feature_set_name]
    return df[cols].to_numpy(), cols

X_train_top50, top50_cols = get_X(train_fe, "top50")
X_test_top50, _           = get_X(test_fe,  "top50")

print(X_train_top50.shape, X_test_top50.shape)


# Training baseline GBM
gbm_baseline = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    random_state=1916,
)

gbm_baseline.fit(X_train_top50, y_train)

gbm_importance = pd.Series(
    gbm_baseline.feature_importances_,
    index=top50_cols
).sort_values(ascending=False)

print("\nGBM-based feature importance (Top 15):")
display(gbm_importance.head(15))

# Build a feature subset from GBM feature importance (top 30)
feature_sets["gbm_top30"] = gbm_importance.head(30).index.tolist()

print("\nAll feature sets after adding gbm_top30:")
for name, cols in feature_sets.items():
    print(f"{name:10s}: {len(cols)} features")


(3993, 50) (363, 50)

GBM-based feature importance (Top 15):


etr_y_ma         0.268089
etr_y_past       0.107655
diff             0.077339
diff_ma          0.057650
WB_GDPpc         0.029406
txt              0.029321
rok              0.021904
sale             0.017691
rr_per_sector    0.016883
WB_GDPgrowth     0.016705
dlc              0.016153
diff_dta         0.015526
lev_past         0.015366
ni               0.014054
xrd              0.013665
dtype: float64


All feature sets after adding gbm_top30:
top20     : 20 features
top30     : 30 features
top50     : 50 features
gbm_top30 : 30 features


In [178]:
# Preparing for parameter tuning later
feature_sets_test = ["top20", "top30", "top50", "gbm_top30"]

print("\nFeature sets to be tested later in model tuning:")
print(feature_sets_test)



Feature sets to be tested later in model tuning:
['top20', 'top30', 'top50', 'gbm_top30']


### 4. Hyperparameter Tuning

In [179]:
def make_tscv(n_splits=5):
    return TimeSeriesSplit(n_splits=n_splits)

def debug_tscv(X, n_splits=5):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    for i, (tr, val) in enumerate(tscv.split(X), 1):
        print(f"Fold {i}: train {len(tr)}, valid {len(val)}")


In [180]:
def tune_model_with_grid(
    estimator,
    param_grid,
    X,
    y,
    model_name="model",
    feature_set_name="top50",
    n_splits=5,
):
    print(f"\n=== {model_name} | features: {feature_set_name} ===")

    gs = GridSearchCV(
        estimator=estimator,
        param_grid=param_grid,
        cv=make_tscv(n_splits),
        scoring=rmse_scorer,
        n_jobs=-1,
        verbose=1,
    )
    gs.fit(X, y)

    print("Best params:", gs.best_params_)
    print(f"Best CV RMSE: {-gs.best_score_:.4f}")

    cv_results = pd.DataFrame(gs.cv_results_).sort_values("rank_test_score")
    return gs.best_estimator_, cv_results.reset_index(drop=True)

### 4.1 AdaBoost

In [69]:
base_tree = DecisionTreeRegressor(
    max_depth=3,
    random_state=1916
)

ada_param_grid = {
    "n_estimators": [100, 300, 500],
    "learning_rate": [0.05, 0.1, 0.2],
}


results_ada = {}

for fs_name in feature_sets_test:
    
    X_train_fs, _ = get_feature_matrix(train_fe, fs_name)
    
    print(f"\n========== Tuning AdaBoost on: {fs_name} ==========")
    
    # GridSearchCV Hyperparameter Tuning
    best_model, cv_results = tune_model_with_grid(
        estimator=AdaBoostRegressor(
            estimator=base_tree,
            random_state=1916
        ),
        param_grid=ada_param_grid,
        X=X_train_fs,
        y=y_train,
        model_name="AdaBoost",
        feature_set_name=fs_name,
        n_splits=5,
    )
    
    best_rmse = -cv_results["mean_test_score"].iloc[0]
    
    results_ada[fs_name] = {
        "best_model": best_model,
        "best_cv_rmse": best_rmse,
    }

print("\n========== AdaBoost CV Results ==========")
for fs_name, res in results_ada.items():
    print(f"{fs_name:10s} | Best CV RMSE = {res['best_cv_rmse']:.4f}")

ada_local_champion = min(
    results_ada.items(),
    key=lambda x: x[1]["best_cv_rmse"]
)

print("\n========== AdaBoost Local Champion ==========")
print(f"Feature Set: {ada_local_champion[0]}")
print(f"Best CV RMSE: {ada_local_champion[1]['best_cv_rmse']:.4f}")
print("\nBest Model:\n", ada_local_champion[1]["best_model"])



========== Tuning AdaBoost on: top20 ==========
=== Tuning AdaBoost on feature set: top20 ===
Param grid:
{'n_estimators': [100, 300, 500], 'learning_rate': [0.05, 0.1, 0.2]}
Fitting 5 folds for each of 9 candidates, totalling 45 fits

Best params:
{'learning_rate': 0.05, 'n_estimators': 100}
Best CV RMSE: 0.1537

========== Tuning AdaBoost on: top30 ==========
=== Tuning AdaBoost on feature set: top30 ===
Param grid:
{'n_estimators': [100, 300, 500], 'learning_rate': [0.05, 0.1, 0.2]}
Fitting 5 folds for each of 9 candidates, totalling 45 fits

Best params:
{'learning_rate': 0.05, 'n_estimators': 100}
Best CV RMSE: 0.1526

========== Tuning AdaBoost on: top50 ==========
=== Tuning AdaBoost on feature set: top50 ===
Param grid:
{'n_estimators': [100, 300, 500], 'learning_rate': [0.05, 0.1, 0.2]}
Fitting 5 folds for each of 9 candidates, totalling 45 fits

Best params:
{'learning_rate': 0.05, 'n_estimators': 100}
Best CV RMSE: 0.1528

========== Tuning AdaBoost on: gbm_top30 ==========

### 4.2 GBM

In [70]:
gbm_param_grid = {
    "n_estimators": [200, 400, 600],   
    "learning_rate": [0.05, 0.1],      
    "max_depth": [2, 3, 4],            
    "subsample": [0.8, 1.0],           
    "max_features": ["sqrt", "log2"],  
}

results_gbm = {}

for fs_name in feature_sets_test:
    
    X_train_fs, _ = get_feature_matrix(train_fe, fs_name)
    
    print(f"\n========== Tuning GBM on: {fs_name} ==========")
    
    best_model, cv_results = tune_model_with_grid(
        estimator=GradientBoostingRegressor(random_state=1916),
        param_grid=gbm_param_grid,
        X=X_train_fs,
        y=y_train,
        model_name="GBM",
        feature_set_name=fs_name,
        n_splits=5,
    )
    
    best_rmse = -cv_results["mean_test_score"].iloc[0]
    
    results_gbm[fs_name] = {
        "best_model": best_model,
        "best_cv_rmse": best_rmse,
    }

print("\n========== GBM CV Results ==========")
for fs_name, res in results_gbm.items():
    print(f"{fs_name:10s} | Best CV RMSE = {res['best_cv_rmse']:.4f}")

gbm_local_champion = min(
    results_gbm.items(),
    key=lambda x: x[1]["best_cv_rmse"]
)

print("\n========== GBM Local Champion ==========")
print(f"Feature Set: {gbm_local_champion[0]}")
print(f"Best CV RMSE: {gbm_local_champion[1]['best_cv_rmse']:.4f}")
print("\nBest Model:\n", gbm_local_champion[1]["best_model"])



========== Tuning GBM on: top20 ==========
=== Tuning GBM on feature set: top20 ===
Param grid:
{'n_estimators': [200, 400, 600], 'learning_rate': [0.05, 0.1], 'max_depth': [2, 3, 4], 'subsample': [0.8, 1.0], 'max_features': ['sqrt', 'log2']}
Fitting 5 folds for each of 72 candidates, totalling 360 fits

Best params:
{'learning_rate': 0.05, 'max_depth': 2, 'max_features': 'sqrt', 'n_estimators': 200, 'subsample': 0.8}
Best CV RMSE: 0.1492

========== Tuning GBM on: top30 ==========
=== Tuning GBM on feature set: top30 ===
Param grid:
{'n_estimators': [200, 400, 600], 'learning_rate': [0.05, 0.1], 'max_depth': [2, 3, 4], 'subsample': [0.8, 1.0], 'max_features': ['sqrt', 'log2']}
Fitting 5 folds for each of 72 candidates, totalling 360 fits

Best params:
{'learning_rate': 0.05, 'max_depth': 2, 'max_features': 'log2', 'n_estimators': 200, 'subsample': 1.0}
Best CV RMSE: 0.1486

========== Tuning GBM on: top50 ==========
=== Tuning GBM on feature set: top50 ===
Param grid:
{'n_estimators'

### 4.3 GBM Histogram

In [71]:
hgb_param_grid = {
    "learning_rate": [0.05, 0.1],
    "max_iter": [300, 600],         
    "max_depth": [3, 5],
    "max_leaf_nodes": [31, 63],      
    "l2_regularization": [0.0, 0.1],
    "max_bins": [255, 511],
}

results_hgb = {}

for fs_name in feature_sets_test:
    
    X_train_fs, _ = get_feature_matrix(train_fe, fs_name)
    
    print(f"\n========== Tuning HistGradientBoosting on: {fs_name} ==========")
    
    best_model, cv_results = tune_model_with_grid(
        estimator=HistGradientBoostingRegressor(
            loss="squared_error",     
            early_stopping=True,       
            random_state=1916,
        ),
        param_grid=hgb_param_grid,
        X=X_train_fs,
        y=y_train,
        model_name="HistGBM",
        feature_set_name=fs_name,
        n_splits=5,
    )
    
    best_rmse = -cv_results["mean_test_score"].iloc[0]
    
    results_hgb[fs_name] = {
        "best_model": best_model,
        "best_cv_rmse": best_rmse,
    }

print("\n========== HistGBM CV Results ==========")
for fs_name, res in results_hgb.items():
    print(f"{fs_name:10s} | Best CV RMSE = {res['best_cv_rmse']:.4f}")

hgb_local_champion = min(
    results_hgb.items(),
    key=lambda x: x[1]["best_cv_rmse"]
)

print("\n========== HistGBM Local Champion ==========")
print(f"Feature Set: {hgb_local_champion[0]}")
print(f"Best CV RMSE: {hgb_local_champion[1]['best_cv_rmse']:.4f}")
print("\nBest Model:\n", hgb_local_champion[1]["best_model"])



========== Tuning HistGradientBoosting on: top20 ==========
=== Tuning HistGBM on feature set: top20 ===
Param grid:
{'learning_rate': [0.05, 0.1], 'max_iter': [300, 600], 'max_depth': [None, 3, 5], 'max_leaf_nodes': [31, 63], 'l2_regularization': [0.0, 0.1], 'max_bins': [255, 511]}
Fitting 5 folds for each of 96 candidates, totalling 480 fits

Best params:
{'l2_regularization': 0.0, 'learning_rate': 0.05, 'max_bins': 255, 'max_depth': 3, 'max_iter': 300, 'max_leaf_nodes': 31}
Best CV RMSE: 0.1491

========== Tuning HistGradientBoosting on: top30 ==========
=== Tuning HistGBM on feature set: top30 ===
Param grid:
{'learning_rate': [0.05, 0.1], 'max_iter': [300, 600], 'max_depth': [None, 3, 5], 'max_leaf_nodes': [31, 63], 'l2_regularization': [0.0, 0.1], 'max_bins': [255, 511]}
Fitting 5 folds for each of 96 candidates, totalling 480 fits

Best params:
{'l2_regularization': 0.0, 'learning_rate': 0.05, 'max_bins': 255, 'max_depth': 3, 'max_iter': 300, 'max_leaf_nodes': 31}
Best CV RMSE:

### 4.4 GBoost

In [72]:
xgb_param_grid = {
    "n_estimators": [300, 600],
    "learning_rate": [0.05, 0.1],
    "max_depth": [3, 4],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "reg_lambda": [0.0, 1.0],   
}

results_xgb = {}

for fs_name in feature_sets_test:
    
    X_train_fs, _ = get_feature_matrix(train_fe, fs_name)
    
    print(f"\n========== Tuning XGBoost on: {fs_name} ==========")
    
    best_model, cv_results = tune_model_with_grid(
        estimator=XGBRegressor(
            objective="reg:squarederror",
            tree_method="hist",  
            random_state=1916,
            n_jobs=-1,
        ),
        param_grid=xgb_param_grid,
        X=X_train_fs,
        y=y_train,
        model_name="XGBoost",
        feature_set_name=fs_name,
        n_splits=5,
    )

    best_rmse = -cv_results["mean_test_score"].iloc[0]
    
    results_xgb[fs_name] = {
        "best_model": best_model,
        "best_cv_rmse": best_rmse,
    }

print("\n========== XGBoost CV Results ==========")
for fs_name, res in results_xgb.items():
    print(f"{fs_name:10s} | Best CV RMSE = {res['best_cv_rmse']:.4f}")

xgb_local_champion = min(
    results_xgb.items(),
    key=lambda x: x[1]["best_cv_rmse"]
)

print("\n========== XGBoost Local Champion ==========")
print(f"Feature Set: {xgb_local_champion[0]}")
print(f"Best CV RMSE: {xgb_local_champion[1]['best_cv_rmse']:.4f}")
print("\nBest Model:\n", xgb_local_champion[1]["best_model"])



========== Tuning XGBoost on: top20 ==========
=== Tuning XGBoost on feature set: top20 ===
Param grid:
{'n_estimators': [300, 600], 'learning_rate': [0.05, 0.1], 'max_depth': [3, 4], 'subsample': [0.8, 1.0], 'colsample_bytree': [0.8, 1.0], 'reg_lambda': [0.0, 1.0]}
Fitting 5 folds for each of 64 candidates, totalling 320 fits

Best params:
{'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 300, 'reg_lambda': 1.0, 'subsample': 1.0}
Best CV RMSE: 0.1546

========== Tuning XGBoost on: top30 ==========
=== Tuning XGBoost on feature set: top30 ===
Param grid:
{'n_estimators': [300, 600], 'learning_rate': [0.05, 0.1], 'max_depth': [3, 4], 'subsample': [0.8, 1.0], 'colsample_bytree': [0.8, 1.0], 'reg_lambda': [0.0, 1.0]}
Fitting 5 folds for each of 64 candidates, totalling 320 fits

Best params:
{'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 300, 'reg_lambda': 1.0, 'subsample': 0.8}
Best CV RMSE: 0.1523

========== Tuning XGBoo

### 4.5 LightGBM 

In [75]:
lgb_param_grid = {
    "n_estimators": [200, 400],
    "learning_rate": [0.05, 0.1],
    "max_depth": [1, 3],
    "num_leaves": [31],       
    "subsample": [0.8],
    "colsample_bytree": [0.8],
    "reg_lambda": [0.0, 1.0],
}

results_lgb = {}

for fs_name in feature_sets_test:

    X_train_fs, _ = get_feature_matrix(train_fe, fs_name)
    
    print(f"\n========== Tuning LightGBM (Fast) on: {fs_name} ==========")
    
    best_model, cv_results = tune_model_with_grid(
        estimator=LGBMRegressor(
            objective="regression",
            random_state=1916,
            n_jobs=-1,
            #early_stopping_rounds=50,
        ),
        param_grid=lgb_param_grid,
        X=X_train_fs,
        y=y_train,
        model_name="LightGBM",
        feature_set_name=fs_name,
        n_splits=5,
    )
    
    best_rmse = -cv_results["mean_test_score"].iloc[0]
    
    results_lgb[fs_name] = {
        "best_model": best_model,
        "best_cv_rmse": best_rmse,
    }


print("\n========== Fast LightGBM CV Results ==========")
for fs_name, res in results_lgb.items():
    print(f"{fs_name:10s} | Best CV RMSE = {res['best_cv_rmse']:.4f}")


lgb_local_champion = min(
    results_lgb.items(),
    key=lambda x: x[1]["best_cv_rmse"]
)

print("\n========== Fast LightGBM Local Champion ==========")
print(f"Feature Set: {lgb_local_champion[0]}")
print(f"Best CV RMSE: {lgb_local_champion[1]['best_cv_rmse']:.4f}")
print("\nBest Model:\n", lgb_local_champion[1]["best_model"])



========== Tuning LightGBM (Fast) on: top20 ==========
=== Tuning LightGBM on feature set: top20 ===
Param grid:
{'n_estimators': [200, 400], 'learning_rate': [0.05, 0.1], 'max_depth': [1, 3], 'num_leaves': [31], 'subsample': [0.8], 'colsample_bytree': [0.8], 'reg_lambda': [0.0, 1.0]}
Fitting 5 folds for each of 16 candidates, totalling 80 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002495 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3530
[LightGBM] [Info] Number of data points in the train set: 3993, number of used features: 18
[LightGBM] [Info] Start training from score 0.215034
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] 

### 4.6 CatBoost

In [76]:
cat_param_grid = {
    "depth": [4, 6, 8],
    "learning_rate": [0.05, 0.1],
    "n_estimators": [300, 600],
    "l2_leaf_reg": [1.0, 3.0],
}

results_cat = {}

for fs_name in feature_sets_test:
    
    X_train_fs, _ = get_feature_matrix(train_fe, fs_name)
    
    print(f"\n========== Tuning CatBoost on: {fs_name} ==========")
    
    best_model, cv_results = tune_model_with_grid(
        estimator=CatBoostRegressor(
            loss_function="RMSE",
            random_state=1916,
            verbose=0,       
            thread_count=-1, 
        ),
        param_grid=cat_param_grid,
        X=X_train_fs,
        y=y_train,
        model_name="CatBoost",
        feature_set_name=fs_name,
        n_splits=5,
    )
    
    best_rmse = -cv_results["mean_test_score"].iloc[0]
    
    results_cat[fs_name] = {
        "best_model": best_model,
        "best_cv_rmse": best_rmse,
    }

print("\n========== CatBoost CV Results ==========")
for fs_name, res in results_cat.items():
    print(f"{fs_name:10s} | Best CV RMSE = {res['best_cv_rmse']:.4f}")

cat_local_champion = min(
    results_cat.items(),
    key=lambda x: x[1]["best_cv_rmse"]
)

print("\n========== CatBoost Local Champion ==========")
print(f"Feature Set: {cat_local_champion[0]}")
print(f"Best CV RMSE: {cat_local_champion[1]['best_cv_rmse']:.4f}")
print("\nBest Model:\n", cat_local_champion[1]["best_model"])



========== Tuning CatBoost on: top20 ==========
=== Tuning CatBoost on feature set: top20 ===
Param grid:
{'depth': [4, 6, 8], 'learning_rate': [0.05, 0.1], 'n_estimators': [300, 600], 'l2_leaf_reg': [1.0, 3.0]}
Fitting 5 folds for each of 24 candidates, totalling 120 fits

Best params:
{'depth': 6, 'l2_leaf_reg': 3.0, 'learning_rate': 0.05, 'n_estimators': 300}
Best CV RMSE: 0.1495

========== Tuning CatBoost on: top30 ==========
=== Tuning CatBoost on feature set: top30 ===
Param grid:
{'depth': [4, 6, 8], 'learning_rate': [0.05, 0.1], 'n_estimators': [300, 600], 'l2_leaf_reg': [1.0, 3.0]}
Fitting 5 folds for each of 24 candidates, totalling 120 fits

Best params:
{'depth': 8, 'l2_leaf_reg': 3.0, 'learning_rate': 0.05, 'n_estimators': 300}
Best CV RMSE: 0.1475

========== Tuning CatBoost on: top50 ==========
=== Tuning CatBoost on feature set: top50 ===
Param grid:
{'depth': [4, 6, 8], 'learning_rate': [0.05, 0.1], 'n_estimators': [300, 600], 'l2_leaf_reg': [1.0, 3.0]}
Fitting 5 fol

In [181]:
catbest_fs_name = cat_local_champion[0]

selected_features = feature_sets[catbest_fs_name]

print("CatBoost uses features:", catbest_fs_name)
print("Number of features:", len(selected_features))


CatBoost uses features: top50
Number of features: 50


### 4.7 XGBoostLSS

In [182]:
if XGBLSS_AVAILABLE:

    print("\n[XGBoostLSS] Library available. Running a simple Gaussian regression template.")

    fs_name = "top50"
    X_train_fs, _ = get_feature_matrix(train_fe, fs_name)

    dtrain = xgb.DMatrix(X_train_fs, label=y_train)

    gauss_dist = Gaussian(
        stabilization="None",
        response_fn="exp",
        loss_fn="nll",
    )

    xgblss = XGBoostLSS(gauss_dist)

    # Simple param_dict
    param_dict = {
    "eta":              ["float", {"low": 0.01,  "high": 0.3,  "log": True}],
    "max_depth":        ["int",   {"low": 2,     "high": 5,    "log": False}],
    "gamma":            ["float", {"low": 1e-8,  "high": 10.0, "log": True}],
    "subsample":        ["float", {"low": 0.7,   "high": 1.0,  "log": False}],
    "colsample_bytree": ["float", {"low": 0.7,   "high": 1.0,  "log": False}],
    "min_child_weight": ["float", {"low": 1e-3,  "high": 10.0, "log": True}],
    "booster":          ["categorical", ["gbtree"]],
}

    # Optuna
    np.random.seed(1916)
    opt_param = xgblss.hyper_opt(
        param_dict,
        dtrain,
        num_boost_round=200,
        nfold=3,
        early_stopping_rounds=20,
        max_minutes=5,
        n_trials=20,
        silence=True,
        seed=1916,
        hp_seed=1916,
    )

    opt_params = opt_param.copy()
    n_rounds = opt_params["opt_rounds"]
    del opt_params["opt_rounds"]

    xgblss.train(opt_params, dtrain, num_boost_round=n_rounds)

    print("[XGBoostLSS] Training finished on top50 features.")



[XGBoostLSS] Library available. Running a simple Gaussian regression template.


  0%|          | 0/20 [00:00<?, ?it/s]

[W 2025-11-23 19:50:54,626] Trial 0 failed with parameters: {'eta': 0.018901928194190506, 'max_depth': 2, 'gamma': 6.254768472566805, 'subsample': 0.7384736733096031, 'colsample_bytree': 0.8740087915828982, 'min_child_weight': 0.001229790585059218, 'booster': 'gbtree'} because of the following error: ModuleNotFoundError('\nCould not find `optuna-integration` for `xgboost`.\nPlease run `pip install optuna-integration[xgboost]`.').
Traceback (most recent call last):
  File "C:\Users\achen\AppData\Roaming\Python\Python312\site-packages\optuna\integration\xgboost.py", line 5, in <module>
    from optuna_integration.xgboost import XGBoostPruningCallback
ModuleNotFoundError: No module named 'optuna_integration'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\achen\AppData\Roaming\Python\Python312\site-packages\optuna\integration\__init__.py", line 132, in _get_module
    return importlib.import_module("." + module_name

ModuleNotFoundError: 
Could not find `optuna-integration` for `xgboost`.
Please run `pip install optuna-integration[xgboost]`.

In [183]:
local_champions = []

local_champions.append({
    "model": "AdaBoost",
    "feature_set": ada_local_champion[0],
    "cv_rmse": ada_local_champion[1]["best_cv_rmse"],
    "estimator": ada_local_champion[1]["best_model"],
})

local_champions.append({
    "model": "GBM",
    "feature_set": gbm_local_champion[0],
    "cv_rmse": gbm_local_champion[1]["best_cv_rmse"],
    "estimator": gbm_local_champion[1]["best_model"],
})

local_champions.append({
    "model": "HistGBM",
    "feature_set": hgb_local_champion[0],
    "cv_rmse": hgb_local_champion[1]["best_cv_rmse"],
    "estimator": hgb_local_champion[1]["best_model"],
})

local_champions.append({
    "model": "XGBoost",
    "feature_set": xgb_local_champion[0],
    "cv_rmse": xgb_local_champion[1]["best_cv_rmse"],
    "estimator": xgb_local_champion[1]["best_model"],
})

local_champions.append({
    "model": "LightGBM",
    "feature_set": lgb_local_champion[0],
    "cv_rmse": lgb_local_champion[1]["best_cv_rmse"],
    "estimator": lgb_local_champion[1]["best_model"],
})

local_champions.append({
    "model": "CatBoost",
    "feature_set": cat_local_champion[0],
    "cv_rmse": cat_local_champion[1]["best_cv_rmse"],
    "estimator": cat_local_champion[1]["best_model"],
})

champions_df = pd.DataFrame(local_champions).sort_values("cv_rmse").reset_index(drop=True)
champions_df


,model,feature_set,cv_rmse,estimator
0,CatBoost,top50,0.146669,<catboost.core.CatBoostRegressor object at 0x0...
1,GBM,top50,0.148377,([DecisionTreeRegressor(criterion='friedman_ms...
2,LightGBM,top30,0.148377,"LGBMRegressor(colsample_bytree=0.8, learning_r..."
3,HistGBM,top30,0.148495,HistGradientBoostingRegressor(early_stopping=T...
4,XGBoost,gbm_top30,0.151412,"XGBRegressor(base_score=None, booster=None, ca..."
5,AdaBoost,gbm_top30,0.152633,"(DecisionTreeRegressor(max_depth=3, random_sta..."


In [85]:
import os

models_dir = "../models/"
os.makedirs(models_dir, exist_ok=True)

print("Model folder ready:", models_dir)


Model folder ready: ../models/


In [86]:
best_models = {
    "AdaBoost":     ada_local_champion[1]["best_model"],
    "GBM":          gbm_local_champion[1]["best_model"],
    "HistGBM":      hgb_local_champion[1]["best_model"],
    "XGBoost":      xgb_local_champion[1]["best_model"],
    "LightGBM":     lgb_local_champion[1]["best_model"],
    "CatBoost":     cat_local_champion[1]["best_model"],
}

best_models


{'AdaBoost': AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=3,
                                                   random_state=1916),
                   learning_rate=0.05, n_estimators=100, random_state=1916),
 'GBM': GradientBoostingRegressor(learning_rate=0.05, max_depth=2, max_features='log2',
                           n_estimators=200, random_state=1916),
 'HistGBM': HistGradientBoostingRegressor(early_stopping=True, learning_rate=0.05,
                               max_depth=3, max_iter=300, random_state=1916),
 'XGBoost': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshol

In [87]:
import pickle

for model_name, model_obj in best_models.items():
    file_path = os.path.join(models_dir, f"{model_name}_best.pkl")

    with open(file_path, "wb") as f:
        pickle.dump(model_obj, f)

    print(f"Saved: {file_path}")


Saved: ../models/AdaBoost_best.pkl
Saved: ../models/GBM_best.pkl
Saved: ../models/HistGBM_best.pkl
Saved: ../models/XGBoost_best.pkl
Saved: ../models/LightGBM_best.pkl
Saved: ../models/CatBoost_best.pkl


# Model Evaluation

### Load Data

In [121]:
base_url = "https://raw.githubusercontent.com/anlan-chen/Gradient_Boosting_Task/main/Data/"

train_fe_url     = base_url + "train_fe.csv"
test_fe_url      = base_url + "test_fe.csv"
feature_rank_url = base_url + "feature_ranking.xlsx"

train_fe = pd.read_csv(train_fe_url)
test_fe  = pd.read_csv(test_fe_url)

train_fe = train_fe.sort_values(by="rok").reset_index(drop=True)
test_fe  = test_fe.sort_values(by="rok").reset_index(drop=True)

print("train_fe shape:", train_fe.shape)
print("test_fe  shape:", test_fe.shape)


train_fe shape: (3993, 116)
test_fe  shape: (363, 116)


In [184]:
feature_rank = pd.read_excel(feature_rank_url, index_col=0)

feature_rank_sorted = feature_rank.sort_values(by="boruta_rank", ascending=True)
feature_names_sorted = feature_rank_sorted.index.tolist()

feature_sets = {
    "top20": feature_names_sorted[:20],
    "top30": feature_names_sorted[:30],
    "top50": feature_names_sorted[:50],
}

for name, cols in feature_sets.items():
    print(f"{name}: {len(cols)} features")


top20: 20 features
top30: 30 features
top50: 50 features


In [129]:
target_col = "etr"

X_train = train_fe[ feature_sets["top50"] ].values
X_test  = test_fe[ feature_sets["top50"] ].values
y_train = train_fe[target_col].values
y_test  = test_fe[target_col].values

print(X_train.shape, X_test.shape)


(3993, 50) (363, 50)


### Load boosting models

In [185]:
def load_pickle_from_url(url):
    response = requests.get(url)
    response.raise_for_status()   
    return pickle.loads(response.content)

model_base = "https://raw.githubusercontent.com/anlan-chen/Gradient_Boosting_Task/main/models/"

model_files = {
    "AdaBoost":    "AdaBoost_best.pkl",
    "GBM":         "GBM_best.pkl",
    "HistGBM":     "HistGBM_best.pkl",
    "XGBoost":     "XGBoost_best.pkl",
    "LightGBM":    "LightGBM_best.pkl",
    "CatBoost":    "CatBoost_best.pkl",
}

loaded_models = {}
for name, fname in model_files.items():
    url = model_base + fname
    obj = load_pickle_from_url(url)
    loaded_models[name] = obj
    print(f"Loaded: {name} from {url}")


Loaded: AdaBoost from https://raw.githubusercontent.com/anlan-chen/Gradient_Boosting_Task/main/models/AdaBoost_best.pkl
Loaded: GBM from https://raw.githubusercontent.com/anlan-chen/Gradient_Boosting_Task/main/models/GBM_best.pkl
Loaded: HistGBM from https://raw.githubusercontent.com/anlan-chen/Gradient_Boosting_Task/main/models/HistGBM_best.pkl
Loaded: XGBoost from https://raw.githubusercontent.com/anlan-chen/Gradient_Boosting_Task/main/models/XGBoost_best.pkl
Loaded: LightGBM from https://raw.githubusercontent.com/anlan-chen/Gradient_Boosting_Task/main/models/LightGBM_best.pkl
Loaded: CatBoost from https://raw.githubusercontent.com/anlan-chen/Gradient_Boosting_Task/main/models/CatBoost_best.pkl


In [168]:
feature_list_by_ranking = feature_names_sorted[:30]


In [169]:
for name, bundle in loaded_models.items():
    bundle["features"] = feature_list_by_ranking


### Baseline Models: OLS, KNN, SVR (top 30)

In [188]:
baseline_scores = {}

# OLS
ols = LinearRegression()
ols.fit(X_train_top30, y_train)
y_pred_ols = ols.predict(X_test_top30)

rmse_ols = rmse(y_test, y_pred_ols)
mae_ols  = mean_absolute_error(y_test, y_pred_ols)

baseline_scores["OLS"] = pd.DataFrame([[rmse_ols, mae_ols]],
                                      columns=["RMSE", "MAE"],
                                      index=["OLS"])

print(f"\nOLS | RMSE={rmse_ols:.6f} | MAE={mae_ols:.6f}")


# KNN 
knn_model = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsRegressor(n_neighbors=5))
])

knn_model.fit(X_train_top30, y_train)
y_pred_knn = knn_model.predict(X_test_top30)

rmse_knn = rmse(y_test, y_pred_knn)
mae_knn  = mean_absolute_error(y_test, y_pred_knn)

baseline_scores["KNN"] = pd.DataFrame([[rmse_knn, mae_knn]],
                                      columns=["RMSE", "MAE"],
                                      index=["KNN"])

print(f"\nKNN | RMSE={rmse_knn:.6f} | MAE={mae_knn:.6f}")


# SVR
svr_model = Pipeline([
    ("scaler", StandardScaler()),
    ("svr", SVR(C=1.0, epsilon=0.1, kernel="rbf"))
])

svr_model.fit(X_train_top30, y_train)
y_pred_svr = svr_model.predict(X_test_top30)

rmse_svr = rmse(y_test, y_pred_svr)
mae_svr  = mean_absolute_error(y_test, y_pred_svr)

baseline_scores["SVR"] = pd.DataFrame([[rmse_svr, mae_svr]],
                                      columns=["RMSE", "MAE"],
                                      index=["SVR"])

print(f"\nSVR | RMSE={rmse_svr:.6f} | MAE={mae_svr:.6f}")


baseline_model_score = pd.concat(baseline_scores.values())
print("\nBaseline models (OLS / KNN / SVR) on TEST set:")
display(baseline_model_score)

# Load boosting models
def load_pickle_from_url(url):
    resp = requests.get(url)
    resp.raise_for_status()
    return pickle.loads(resp.content)

model_base = base_url + "models/"

model_files = {
    "AdaBoost": "AdaBoost_best.pkl",
    "GBM":      "GBM_best.pkl",
    "HistGBM":  "HistGBM_best.pkl",
    "XGBoost":  "XGBoost_best.pkl",
    "LightGBM": "LightGBM_best.pkl",
    "CatBoost": "CatBoost_best.pkl",
}



OLS | RMSE=0.084113 | MAE=0.058745

KNN | RMSE=0.095373 | MAE=0.068674

SVR | RMSE=0.084942 | MAE=0.061764

Baseline models (OLS / KNN / SVR) on TEST set:


,RMSE,MAE
OLS,0.084113,0.058745
KNN,0.095373,0.068674
SVR,0.084942,0.061764


In [190]:
import re

numeric_cols = train_fe.select_dtypes(include=[np.number]).columns.drop("etr")

scores = []

for name, bundle in loaded_models.items():
    model = bundle["model"]
    fs_name = bundle["feature_set_name"]

    m = re.search(r"top(\d+)", fs_name)
    k = int(m.group(1)) if m else 30

    feats = numeric_cols[:k]
    X_test_fs = test_fe[feats].to_numpy()

    pred = model.predict(X_test_fs)

    rmse_val = rmse(y_test, pred)
    mae_val  = mean_absolute_error(y_test, pred)

    print(f"{name:10s} | {fs_name:10s} | k={k:3d} → RMSE={rmse_val:.6f}, MAE={mae_val:.6f}")

    scores.append([name, rmse_val, mae_val])


boosting_model_score = pd.DataFrame(scores, columns=["Model", "RMSE", "MAE"]).set_index("Model")
display(boosting_model_score)


AdaBoost   | gbm_top30  | k= 30 → RMSE=0.156998, MAE=0.136897
GBM        | top50      | k= 50 → RMSE=0.681585, MAE=0.653138
HistGBM    | top30      | k= 30 → RMSE=0.314693, MAE=0.300492
XGBoost    | gbm_top30  | k= 30 → RMSE=0.913832, MAE=0.905455
LightGBM   | top30      | k= 30 → RMSE=0.290798, MAE=0.276403
CatBoost   | top50      | k= 50 → RMSE=0.175423, MAE=0.155062


,RMSE,MAE
Model,,
AdaBoost,0.156998,0.136897
GBM,0.681585,0.653138
HistGBM,0.314693,0.300492
XGBoost,0.913832,0.905455
LightGBM,0.290798,0.276403
CatBoost,0.175423,0.155062


In [191]:
final_table = pd.concat([
    baseline_model_score,
    boosting_model_score
])

print("\nFINAL MODEL COMPARISON")
display(final_table)

champion = final_table.sort_values("RMSE").iloc[0]

print("\nBEST MODEL (CHAMPION)")
print(champion)



FINAL MODEL COMPARISON


,RMSE,MAE
OLS,0.084113,0.058745
KNN,0.095373,0.068674
SVR,0.084942,0.061764
AdaBoost,0.156998,0.136897
GBM,0.681585,0.653138
HistGBM,0.314693,0.300492
XGBoost,0.913832,0.905455
LightGBM,0.290798,0.276403
CatBoost,0.175423,0.155062



BEST MODEL (CHAMPION)
RMSE    0.084113
MAE     0.058745
Name: OLS, dtype: float64


The simplest model, Ordinary Least Squares (OLS), achieved the best predictive performance, with the lowest RMSE and MAE among all models.